<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/notebook/chapter4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1次元非正規分布の最尤推定による異常検知
- ホテリング理論による異常検知手法は、データが正規分布に従うことを前提としている
- 正規分布以外の1次元確率分布を用いた異常検知の手法

## A. データの分布形状から確率分布の種類を選択する
適用する確率分布の種類を選択する

データの分布の形状に注目する
- 分布の山の数
- 分布の左右対称性
- 分布の裾の重さ

＋AICやBICといった定量的な評価指標を組み合わせる

### 分布の山の数

正規分布や対数正規分布のような主要な確率分布の多くは、山が一つの形状（単峰性）を持つ。そのため、データを可視化した際にヒストグラムで山が2つ以上ある場合は単峰性の分布をそのまま適用できない。

### 分布の左右対称性

分布形状が左右対称化は、確率分布の選択に大きく影響を与える。具体的には、左右対称であれば正規分布やスチューデントのt分布が、左右非対称であれば対数正規分布やガンマ分布が有力な候補になる。定量的に評価するためには平均値と中央値の差や、歪度が利用できる

### 分布の裾の重さ

裾の重い分布（heavy-tailed distribution）とは、分布端部での確率密度の減衰が緩やかで、外れ値が多く出現する分布を指す。定量的に評価するためには、尖度が利用できる。正規分布よりもt分布の方が、ガンマ分布よりも対数正規分布の方が裾が重いため、裾の重さに応じてこれらの分布を絞り込むこともできる